In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

In [2]:
stores_train = pd.read_csv('../../data/raw/stores_train.csv')
data_age = pd.read_csv('../../data/raw/grunnkrets_age_distribution.csv')
data_income = pd.read_csv('../../data/raw/grunnkrets_income_households.csv')
data_geography = pd.read_csv('../../data/raw/grunnkrets_norway_stripped.csv') 

In [3]:
def average_revenue_of_chain(dataset_stores):
    "Average revenue of chains in datasett"
    dataset_stores = dataset_stores[(dataset_stores["year"] == 2016)]
    
    return dataset_stores.groupby(['chain_name'])['revenue'].mean()
average_revenue_of_chain(stores_train)

chain_name
3T                    14.179875
AB KLIPP               5.973500
ACE COLLECTION         2.969667
ADAM OG EVA FRISØR     4.208200
ALEX SUSHI            15.575000
                        ...    
ZIZZI                  4.051143
ZOO 1 GRUPPEN          3.505231
ZOOKJEDEN BUDDY        5.296800
ÅPENT BAKERI           5.963000
ØDEGAARD DAME          1.000000
Name: revenue, Length: 306, dtype: float64

In [4]:
def average_revenue_of_mall(dataset_stores):
    "Average revenue of malls in dataset"
    dataset_stores = dataset_stores[(dataset_stores["year"] == 2016)]
    return dataset_stores.groupby(['mall_name'])['revenue'].mean()
average_revenue_of_mall(stores_train)

mall_name
ABC Stormarked        1.468000
Aker Brygge          15.949500
Aksdal Senter         2.721286
Alna Senter          61.234000
Amanda Storsenter     5.030714
                       ...    
Østbanehallen        55.917000
Østerås Senter        4.215500
Østfoldhallene        6.465364
Øyrane Torg           6.160636
Øysenteret            0.128000
Name: revenue, Length: 487, dtype: float64

In [5]:
def population(dataset_age): 
    age_df = dataset_age[(dataset_age["year"] == 2016)]
    population = age_df.drop(["grunnkrets_id", "year"], axis=1).sum(axis=1)
    age_df["population_count"] = population 
    return age_df[["grunnkrets_id", "population_count"]]
population(data_age)

<ipython-input-5-40e8a7f334f2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_df["population_count"] = population


,grunnkrets_id,population_count
1,16013117,944
3,16011203,880
4,3011601,645
8,3010807,528
11,3012307,876
...,...,...
22610,9380111,81
22612,9400101,78
22615,9400104,84
22618,9400108,151


In [6]:
def population_grouped(data_age, data_geography, grouping_element): 
    age_df = population(data_age)
    geography_df = data_geography[data_geography["year"] == 2016]
    population_df = age_df.merge(geography_df, how="left", on ="grunnkrets_id")
    grouped_df = population_df.groupby([grouping_element], as_index = False)["population_count"].sum()
    return grouped_df

In [7]:
def mean_income_per_capita(dataset_age,dataset_income):
    "mean income per capita per grunnkrets"
    age_df = population(data_age)
    income_df = dataset_income[dataset_income["year"] == 2016]
    age_and_income_df = age_df.merge(income_df, how='left', on='grunnkrets_id')
    mean_income = age_and_income_df.drop(['year','singles','couple_without_children','couple_with_children','other_households','single_parent_with_children'],axis=1)
    mean_income['mean_income']=mean_income['all_households']/mean_income['population_count']
    mean_income=mean_income.drop(['all_households'], axis=1)

    return mean_income.he

    
mean_income_per_capita(data_age,data_income)
    

<ipython-input-5-40e8a7f334f2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_df["population_count"] = population


AttributeError: 'DataFrame' object has no attribute 'he'

In [ ]:
def mean_income_per_capita_grouped(dataset_income,dataset_geography,grouping_element):
    #gets data from mean_income_per_capita functino
    data_mean_income = mean_income_per_capita(data_age,dataset_income)
    #gets data from geography set and makes sure we only use data for 2016
    geography_df = dataset_geography[dataset_geography["year"] == 2016]
    #gets the data of mean income with the geography data
    mean_income_geo_df = data_mean_income.merge(geography_df, how='left', on='grunnkrets_id')
    #sum the number of people based on grouping element
    grouped_population_df=mean_income_geo_df.groupby([grouping_element], as_index = False)["population_count"].sum()
    #merge this with the grunnkrets to see both total population per selected area and grunnkrets
    total_grouped_df=mean_income_geo_df.merge(grouped_population_df,how='left',on=grouping_element)
    portion_income_df=total_grouped_df
    #find ration of grunnkrets to total population and multiply this with grunnkrets mean income
    portion_income_df['mean_income']=total_grouped_df['mean_income']*total_grouped_df['population_count_x']/total_grouped_df['population_count_y']
    #add these incomes together, should add up to the total mean income for the selected area
    grouped_income_df=portion_income_df.groupby([grouping_element], as_index = False)["mean_income"].sum() 
    return grouped_income_df.head(10)

mean_income_per_capita_grouped(data_income,data_geography,'grunnkrets_id')

<ipython-input-5-40e8a7f334f2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_df["population_count"] = population


AttributeError: 'DataFrame' object has no attribute 'he'

In [9]:
stores_train

,store_id,year,store_name,plaace_hierarchy_id,sales_channel_name,grunnkrets_id,address,lat,lon,chain_name,mall_name,revenue,geometry
0,983540538-974187930-44774,2016,MCDONALD'S BRAGERNES TORG MAGASINET,1.1.1.0,Hamburger restaurants,6020303,BRAGERNES TORG 13,59.743104,10.204928,MCDONALDS,Magasinet Drammen,17.998,POINT (10.204927885979 59.743104058362704)
1,987074191-973117734-44755,2016,MCDONALD'S KLINGENBERGGATA,1.1.1.0,Hamburger restaurants,3010306,NaN,59.913759,10.734031,MCDONALDS,NaN,23.828,POINT (10.7340307646896 59.9137594158249)
2,984890265-981157303-64491,2016,BURGER KING HØNEFOSS,1.1.1.0,Hamburger restaurants,6050102,KONG RINGS GATE 1,60.164751,10.254656,BURGER KING,Kuben Hønefoss,16.099,POINT (10.2546558936519 60.1647505705996)
3,914057442-992924179-126912,2016,BURGER KING GLASSHUSPASSASJEN,1.1.1.0,Hamburger restaurants,18040102,STORGATA 12,67.283669,14.379796,BURGER KING,Glasshuspassasjen,9.296,POINT (14.379796237766 67.28366871498261)
4,913018583-913063538-668469,2016,BURGER KING TILLERTORGET,1.1.1.0,Hamburger restaurants,16017414,NaN,63.358068,10.374832,BURGER KING,Tillertorget,4.528,POINT (10.374832468995699 63.3580678289847)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12854,915789943-915806929-781991,2016,MEIERIGÅRDEN BRYGGERIUTSALG,2.8.11.2,Beer and soda shop,7010705,THUEGATA 2,59.416276,10.480970,NaN,NaN,0.088,POINT (10.4809697539876 59.41627592125261)
12855,917921733-917982368-868081,2016,GULATING ØLUTSALG CC GJØVIK,2.8.11.2,Beer and soda shop,5020406,NaN,60.799991,10.693635,GULATING GRUPPEN,CC Gjøvik,1.816,POINT (10.693634816499099 60.799990685550696)
12856,911721961-911764474-496764,2016,GULATING ØLUTSALG STRØMMEN,2.8.11.2,Beer and soda shop,2310803,STØPERIVEIEN 6,59.946562,11.007659,GULATING GRUPPEN,Strømmen Storsenter,38.225,POINT (11.007659406372099 59.9465624207024)
12857,914337046-914343372-721294,2016,DET GODE BRYGG,2.8.11.2,Beer and soda shop,11020113,VÅGSGATA 16,58.850261,5.735674,NaN,Bystasjonen,3.642,POINT (5.73567433152869 58.8502613611896)


In [11]:
%pip install shapely
from shapely.geometry import Point

def make_point(stores_train):
    stores_train['geometry'] = [Point(xy) for xy in zip(stores_train.lon, stores_train.lat)] 
    stores_train_new = stores_train[['store_id','geometry']]
    return stores_train_new

make_point(stores_train)

Note: you may need to restart the kernel to use updated packages.


,store_id,geometry
0,983540538-974187930-44774,POINT (10.204927885979 59.743104058362704)
1,987074191-973117734-44755,POINT (10.7340307646896 59.9137594158249)
2,984890265-981157303-64491,POINT (10.2546558936519 60.1647505705996)
3,914057442-992924179-126912,POINT (14.379796237766 67.28366871498261)
4,913018583-913063538-668469,POINT (10.374832468995699 63.3580678289847)
...,...,...
12854,915789943-915806929-781991,POINT (10.4809697539876 59.41627592125261)
12855,917921733-917982368-868081,POINT (10.693634816499099 60.799990685550696)
12856,911721961-911764474-496764,POINT (11.007659406372099 59.9465624207024)
12857,914337046-914343372-721294,POINT (5.73567433152869 58.8502613611896)
